# Assignment 3 - Ensemble Methods #

Welcome to your third assignment. This exercise will test your understanding on Ensemble Methods.

In [17]:
# Always run this cell
import numpy as np
import pandas as pd

# USE THE FOLLOWING RANDOM STATE FOR YOUR CODE
RANDOM_STATE = 42

## Download the Dataset ##
Download the dataset using the following cell or from this [link](https://github.com/sakrifor/public/tree/master/machine_learning_course/EnsembleDataset) and put the files in the same folder as the .ipynb file. 
In this assignment you are going to work with a dataset originated from the [ImageCLEFmed: The Medical Task 2016](https://www.imageclef.org/2016/medical) and the **Compound figure detection** subtask. The goal of this subtask is to identify whether a figure is a compound figure (one image consists of more than one figure) or not. The train dataset consits of 4197 examples/figures and each figure has 4096 features which were extracted using a deep neural network. The *CLASS* column represents the class of each example where 1 is a compoung figure and 0 is not. 


In [18]:
import urllib.request
url_train = 'https://github.com/sakrifor/public/raw/master/machine_learning_course/EnsembleDataset/train_set.csv'
filename_train = 'train_set.csv'
urllib.request.urlretrieve(url_train, filename_train)
url_test = 'https://github.com/sakrifor/public/raw/master/machine_learning_course/EnsembleDataset/test_set_noclass.csv'
filename_test = 'test_set_noclass.csv'
urllib.request.urlretrieve(url_test, filename_test)

('test_set_noclass.csv', <http.client.HTTPMessage at 0x7fbc50043590>)

In [19]:
# Run this cell to load the data
train_set = pd.read_csv("train_set.csv").sample(frac=1).reset_index(drop=True)
train_set.head()
X = train_set.drop(columns=['CLASS'])
y = train_set['CLASS'].values

## 1.0 Testing different ensemble methods ##
In this part of the assignment you are asked to create and test different ensemble methods using the train_set.csv dataset. You should use **10-fold cross validation** for your tests and report the average f-measure and accuracy of your models.

### !!! Use n_jobs=-1 where is posibble to use all the cores of a machine for running your tests ###

### 1.1 Voting ###
Create a voting classifier which uses three estimators/classifiers. Test both soft and hard voting and choose the best one.

In [20]:
# BEGIN CODE HERE
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import KFold
from sklearn.linear_model import SGDClassifier

cls1 = DecisionTreeClassifier(min_samples_leaf=100, min_impurity_decrease=0.001) # Classifier #1 
cls2 = SGDClassifier() # Classifier #2 
cls3 = ExtraTreeClassifier() #Classifier #3
vcls = VotingClassifier([('dt1', cls1), ('dt2', cls2), ('dt3', cls3)], n_jobs=-1, voting="hard") # Voting Classifier

kf = KFold(n_splits=10)
accuracies = []
fmeasures = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    pred = vcls.fit(X_train,y_train).predict(X_test)
    accuracies.append(accuracy_score(y_test,pred))
    fmeasures.append(f1_score(y_test,pred))
    
avg_fmeasure = sum(fmeasures)/len(fmeasures) # The average f-measure
avg_accuracy = sum(accuracies)/len(accuracies) # The average accuracy

#END CODE HERE

In [21]:
print("Classifier:")
print(vcls)
print("F1-Score:{} & Accuracy:{}".format(avg_fmeasure,avg_accuracy))

Classifier:
VotingClassifier(estimators=[('dt1',
                              DecisionTreeClassifier(min_impurity_decrease=0.001,
                                                     min_samples_leaf=100)),
                             ('dt2', SGDClassifier()),
                             ('dt3', ExtraTreeClassifier())],
                 n_jobs=-1)
F1-Score:0.8296609646646731 & Accuracy:0.7962330946698488


### 1.2 Stacking ###
Create a stacking classifier which uses two estimators/classifiers. Try different classifiers for the combination of the initial classifiers. Report your results in the following cell.

In [22]:
# BEGIN CODE HERE
from sklearn.ensemble import StackingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, accuracy_score

cls1 = DecisionTreeClassifier(max_depth=5,splitter = "best",min_impurity_decrease=3e-6) # Classifier #1 
cls2 = DecisionTreeClassifier(max_depth=7,splitter = "random",min_impurity_decrease=3e-6) # Classifier #2 
scls = StackingClassifier([('dt1',cls1),('dt2',cls2)], n_jobs=-1) # Stacking Classifier

kf = KFold(n_splits=10)
accuracies = []
fmeasures = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    pred = scls.fit(X_train,y_train).predict(X_test)
    accuracies.append(accuracy_score(y_test,pred))
    fmeasures.append(f1_score(y_test,pred))

avg_fmeasure = sum(fmeasures)/len(fmeasures) # The average f-measure
avg_accuracy = sum(accuracies)/len(accuracies) # The average accuracy

#END CODE HERE

In [23]:
print("Classifier:")
print(scls)
print("F1-Score:{} & Accuracy:{}".format(avg_fmeasure,avg_accuracy))

Classifier:
StackingClassifier(estimators=[('dt1',
                                DecisionTreeClassifier(max_depth=5,
                                                       min_impurity_decrease=3e-06)),
                               ('dt2',
                                DecisionTreeClassifier(max_depth=7,
                                                       min_impurity_decrease=3e-06,
                                                       splitter='random'))],
                   n_jobs=-1)
F1-Score:0.7817459479093924 & Accuracy:0.7295005114217525


### 1.3 Report the results ###  
Report the results of your experiments in the following cell. How did you choose your initial classifiers? 

1.1 Με hard-voting το μοντέλο είχε ελάχιστα καλύτερη απόδοση από ότι με soft-voting χρησιμοποιώντας μόνο DecisionTreeClassifier. Ωστόσο, μετά από δοκιμές με ένα DecisionTreeClassifier, ένα SGDClassifier και ένα ExtraTree έχω ακόμα καλύτερα αποτελέσματα πετυχαίνοντας F1-Score = 0.829 και Accuracy = 0.795, χωρίς όμως να έχω την δυνατότητα για soft-voting, γιατί ο SGDClassifier δεν έχει μέθοδο predict_proba. Γενικότερα φαίνεται η μέθοδος voting-classifier να αποδίδει καλύτερα με διαφορετικούς κατηγοριοποιητές, για αυτό και χρησιμοποιήθηκαν 3 διαφορετικοί, με τον extra-tree να κανει σχεδον την ιδια δουλεια με ενα απλο decision-tree απλά με πιο τυχαίες παραμέτρους. 

1.2
Έγιναν διάφοροι έλεγχοι σε παραμέτρους 2 DecisionTreeClassifiers. Συγκεκριμένα μετρήθηκε η απόδοση για βάθος 5,10,15 στο πρώτο και 3,7,20 στο δεύτερο με όλους τους δυνατούς τους συνδιασμούς. Για τον τρόπο που προχωράει η επιλογή για τον επόμενο κόμβο(splitter best και random) καθώς και για τιμές min_impurity_decrease(1e-6,3e-6,1e-7) καθώς και όλοι οι συνδιασμοί τους. Έτσι δίνοντας στον πρώτο classifier max_depth=5,splitter = "best",min_impurity_decrease=3e-6 και στον δεύτερο max_depth=7,splitter = "random",min_impurity_decrease=3e-6 πετυχαίνει F1-Score = 0.785 και Accuracy = 0.733

## 2.0 Randomization ##

**2.1** You are asked to create three ensembles of decision trees where each one is produced with a different way from the ones discussed in the lecture for producing homogeneous ensembles. Compare them with a simple decision tree classifier and report your results in the dictionaries (dict) below using as key the given name of your classifier and as value the f1/accuracy score. The dictionaries should contain four different elements.  

In [24]:
##### BEGIN CODE HERE
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier


ens1 = BaggingClassifier(
    base_estimator = DecisionTreeClassifier(),
    n_estimators = 10,
    max_features = 0.7,
    n_jobs = -1) #Random Subspaces Method

ens2 = RandomForestClassifier(
    n_estimators = 100,
    max_depth = 15,
    min_samples_split = 50,
    bootstrap = True,
    n_jobs = -1)  #Random Forest

ens3 = AdaBoostClassifier(
    base_estimator = DecisionTreeClassifier(),
    n_estimators = 50) #AdaBoost

tree = DecisionTreeClassifier(min_samples_leaf=200)

names = ['bagging','random_forest','adaboost','simple_tree']
classifiers = [ens1,ens2,ens3,tree]

f_measures = dict()
accuracies = dict()
# Example f_measures = {'Simple Decision':0.8551, 'Ensemble with random ...': 0.92, ...}

for name,classifier in zip(names,classifiers):
    kf = KFold(n_splits=10)
    my_accuracies = []
    my_fmeasures = []

    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]
        pred = classifier.fit(X_train,y_train).predict(X_test)
        my_accuracies.append(accuracy_score(y_test,pred))
        my_fmeasures.append(f1_score(y_test,pred))

    avg_fmeasure = sum(my_fmeasures)/len(my_fmeasures) # The average f-measure
    avg_accuracy = sum(my_accuracies)/len(my_accuracies) # The average accuracy
    f_measures[name] = avg_fmeasure
    accuracies[name] = avg_accuracy
    

#END CODE HERE

In [25]:
print(ens1)
print(ens2)
print(ens3)
print(tree)
for name,score in f_measures.items():
    print("Classifier:{} -  F1:{}".format(name,score))
for name,score in accuracies.items():
    print("Classifier:{} -  Accuracy:{}".format(name,score))

BaggingClassifier(base_estimator=DecisionTreeClassifier(), max_features=0.7,
                  n_jobs=-1)
RandomForestClassifier(max_depth=15, min_samples_split=50, n_jobs=-1)
AdaBoostClassifier(base_estimator=DecisionTreeClassifier())
DecisionTreeClassifier(min_samples_leaf=200)
Classifier:bagging -  F1:0.8049394778173184
Classifier:random_forest -  F1:0.840539783357349
Classifier:adaboost -  F1:0.7419835258308327
Classifier:simple_tree -  F1:0.7668403294932741
Classifier:bagging -  Accuracy:0.7740675076713263
Classifier:random_forest -  Accuracy:0.7976622343448119
Classifier:adaboost -  Accuracy:0.6968428230480737
Classifier:simple_tree -  Accuracy:0.7111552449141947


**2.2** Describe your classifiers and your results.

1.Random Subspaces Method. Δουλεύει με όλα τα παραδείγματα του συνόλου αλλά όχι με όλα τα χαρακτηριστικά αυτού. Δημιουργία 10 DecisionTreeClassifiers κάθε ένα εκ των οποίων εκπαιδεύεται στο 70% των χαρακτηριστικών του συνόλου δεδομένων χρησιμοποιώντας όλους τους πόρους του συστήματος για παραλληλία.
-Μέση ακρίβεια: 76.9%, Μέση τιμή f1Score: 80%

2.RandomForestClassifier. Έχω 100 DecisionTreeClassifiers κάθε ένα από αυτά έχει μέγιστο βάθος 15, και ελάχιστο αριθμό παραδειγμάτων για σπάσιμο κόμβου 50, για αποφυγή overfitting. Χρησιμοποιούνται όλοι οι πυρήνες για την εκπαίδευση και την πρόβλεψη. Κάθε δέντρο εκπαιδεύεται σε κάποιο τυχαίο υποσύνολο του συνόλου δεδομένων, ίδιο μέγεθος με ολόκληρο το σύνολο, με επανατοποθέτηση. Το τελικό αποτέλεσμα προκύπτει από τον συνδιασμό όλων τον προβλέψεων των δέντρων.
-Μέση ακρίβεια: 79.2%, Μέση τιμή f1Score: 83.3%

3.AdaBoostClassifier. Δημιουργία 50 DecisionTreeClassifiers. Το πρώτο δέντρο κάνει fit και predict στα δεδομένα όπως ένα απλό δέντρο. Το δεύτερο δίνει μεγαλύτερο βάρος στα παραδείγματα στα οποία το πρώτο απέτυχε να βρει την σωστή κλάση. Το τρίτο δίνει μεγαλύτερο βάρος σε αυτά που απέτυχε το δεύτερο και ούτω καθεξής. Σε αυτή τη μέθοδο δεν υπάρχει παράλληλη επεξεργασία, καθώς για κάθε δέντρο χρησιμοποιεί το προηγούμενό του.
-Μέση ακρίβεια: 69.7%, Μέση τιμή f1Score: 74%

**2.3** Increasing the number of estimators in a bagging classifier can drastically increase the training time of a classifier. Is there any solution to this problem? Can the same solution be applied to boosting classifiers?

Στην περίπτωση που αυξηθεί πολύ ο αριθμός των classifiers μπορεί να μειωθεί ο χρόνος εκτέλεσης χρησιμοποιώντας όλους τους πυρήνες του συστήματος, μοιράζοντας την δουλειά του κάθε classifier. Αυτό δεν μπορεί να επιτευχθεί σε έναν boosting classifier, καθώς η εκτέλεση του κάθε classifier γίνεται διαδοχικά. Ο κάθε classifier χρησιμοποιεί την έξοδο του προηγούμενού του για να εκτελεστεί.

## 3.0 Creating the best classifier ##

**3.1** In this part of the assignment you are asked to train the best possible ensemble! Describe the process you followed to achieve this result. How did you choose your classifier and your parameters and why. Report the f-measure & accuracy (10-fold cross validation) of your final classifier and results of classifiers you tried in the cell following the code. Can you achieve an accuracy over 83-84%?

In [26]:
# BEGIN CODE HERE
ens1 = RandomForestClassifier(
    n_estimators = 100,
    max_depth = 15,
    n_jobs = -1)
ens2 = SGDClassifier()
ens3 = ExtraTreeClassifier()
ens4 = RandomForestClassifier(
    n_estimators = 50)
best_cls = StackingClassifier([('dt1', ens2), ('dt2', ens3), ('dt3', ens4), ('dt4', ens1)], n_jobs=-1)

kf = KFold(n_splits=10)
best_accuracy = []
best_fmeasure = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    pred = best_cls.fit(X_train,y_train).predict(X_test)
    best_accuracy.append(accuracy_score(y_test,pred))
    best_fmeasure.append(f1_score(y_test,pred))

best_fmeasure = sum(best_fmeasure)/len(best_fmeasure)
best_accuracy = sum(best_accuracy)/len(best_accuracy)

#END CODE HERE

/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [27]:
print("Classifier:")
print(best_cls)
print("F1-Score:{} & Accuracy:{}".format(best_fmeasure,best_accuracy))

Classifier:
StackingClassifier(estimators=[('dt1', SGDClassifier()),
                               ('dt2', ExtraTreeClassifier()),
                               ('dt3', RandomForestClassifier(n_estimators=50)),
                               ('dt4',
                                RandomForestClassifier(max_depth=15,
                                                       n_jobs=-1))],
                   n_jobs=-1)
F1-Score:0.8673898951147857 & Accuracy:0.8403216274576655


###### **3.2** Describe the process you followed to achieve this result. How did you choose your classifier and your parameters and why. Report the f-measure & accuracy (10-fold cross validation) of your final classifier and results of classifiers you tried in the cell following the code.

Έγιναν διάφοροι συνδιασμοί προηγούμενων classifiers για την υλοποιήση του τελικού. Συγκεκριμένα υπολόγισα την ακρίβεια δοκιμάζοντας τόσο με VotingClassifier όσο και με StackingClassifier για διάφορα σύνολα κατηγοριοποιητών. Για παράδειγμα με 3 RandomForest και StackingClassifier με ακρίβεια περίπου 81.5%, με VotingClassifier δεν είχε τόσο καλή απόδοση ενώ με hard voting ήταν αρκετά χειρότερη. Δοκίμασα να βάλω δύο RandomForest μαζί με έναν BaggingClassifier σε έναν VottingClassifier πετυχαίνοντας ακρίβεια γύρω στο 78%. Δοκιμάστηκαν και άλλες παραλλαγές με classifiers όπως ο ExtraTreeClassifier, SGDClassifier μαζί με κάποιο RandomForest και όλα αυτά μέσα σε έναν Voting ή Stacking Classifier.

Καλύτερα αποτελέσματα έδινε ένας StackingClassifier με δύο RandomForestClassifiers, έναν SGDClassifier και έναν ExtraTreeClassifier μέσα σε αυτόν. 

Έπειτα δοκιμάστηκαν κάποιες μεταθέσεις στη σειρά που είχαν οι παραπάνω classifiers μέσα στον Stacking παίρνοντας ελαφρώς διαφορετικά αποτελέσματα. Ο καλύτερος κατηγοριοποιητής που κατέληξα πετυχαίνει F1-Score:0.8688 και Accuracy:0.8436. 

**3.3** Create a classifier that is going to be used in production - in a live system. Use the *test_set_noclass.csv* to make predictions. Store the predictions in a list.  

In [28]:
# BEGIN CODE HERE

# Τελικός κατηγοριοποιητής είναι ο ίδιος που απέδιδε καλύτερα και στο παραπάνω ερώτημα.
ens1 = RandomForestClassifier(
    n_estimators = 100,
    max_depth = 15,
    n_jobs = -1,
    random_state=RANDOM_STATE)
ens2 = SGDClassifier(random_state=RANDOM_STATE)
ens3 = ExtraTreeClassifier(random_state=RANDOM_STATE)
ens4 = RandomForestClassifier(
    n_estimators = 50,
    random_state=RANDOM_STATE)

cls = StackingClassifier([('dt1', ens2), ('dt2', ens3), ('dt3', ens4), ('dt4', ens1)], n_jobs=-1)
test_set = pd.read_csv("test_set_noclass.csv").sample(frac=1).reset_index(drop=True)
predictions = cls.fit(X,y).predict(test_set)

#END CODE HERE

/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [29]:
print(cls)
print(predictions)

StackingClassifier(estimators=[('dt1', SGDClassifier(random_state=42)),
                               ('dt2', ExtraTreeClassifier(random_state=42)),
                               ('dt3',
                                RandomForestClassifier(n_estimators=50,
                                                       random_state=42)),
                               ('dt4',
                                RandomForestClassifier(max_depth=15, n_jobs=-1,
                                                       random_state=42))],
                   n_jobs=-1)
[0 0 0 ... 1 0 1]
